In [1]:
#2000 emails sent to email_history took 5 minutes
%load_ext autoreload
%autoreload 2
import logging
import warnings
import os
import pandas as pd
from config import *
from Scraping_Emails.modules.scrapes import scrape
from Scraping_Emails.modules.db_operations_aws import DatabaseConnector
from Sending_Emails.modules.sends import EmailConfig
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Scraper.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)

# ------------------------------------packaging process into final function----------------------------------------

def process(subject_line, email_address, email_pass, start_date, campaign_name, sport):

    #create the msgs via scrape, then iterate through list of messages, and lastly cleanse frame

    #First thing to occur everytime
    logging.info('\n\nNew logging instance')
 
    msgs_inbox = scrape.scrape_msgs_outbox_or_inbox('inbox', subject_line, email_address, email_pass, start_date, 'Current Subject Line')
    inbox = scrape.create_msg_frame(msgs_inbox)
    inbox = scrape.cleanse_frame(inbox, 'inbox')
    inbox.name = 'inbox'

    msgs_outbox = scrape.scrape_msgs_outbox_or_inbox('outbox', subject_line, email_address, email_pass, start_date, 'Current Subject Line')
    outbox = scrape.create_msg_frame(msgs_outbox)
    outbox = scrape.cleanse_frame(outbox, 'outbox')
    outbox.name = 'outbox'
        
    if inbox.empty == True:
        thread = pd.DataFrame()
        pass
    else: #this is a left merge on outbox
        thread = scrape.piece_together(outbox, inbox)
        thread = scrape.assign_sentiment(thread)
       
    # #If message was accidentally triggered more than once
    thread = thread.drop_duplicates(subset = ['subject', 'to'], keep='last')
    thread.reset_index(drop = True, inplace = True)
    outbox = scrape.map_reply_outbox(thread, outbox)

    #Optional column identifiers to add in to the DB
    outbox['email_campaign_tag'] = campaign_name
    outbox['sport'] = sport    #frame   #table_name

    return(inbox, outbox, thread)


# -------------------------------------------

#Get inbox, outbox, and create thread with given subject line back to given date
inbox, outbox, thread= process(email_config.email_subject_line, email_config.EMAIL_ADDRESS_FROM, email_config.EMAIL_PASS, email_config.filter_date, email_config.email_campaign_name, email_config.sport)

# ----------Send outbox

if outbox.empty != True:
    db_connector_email_history = DatabaseConnector(email_config.server, email_config.db_user, email_config.db_pass, email_config.database, 'email_history')
    db_connector_email_history.send(outbox, 'email_history')
else:
    logging.info('Outbox is an empty frame')
    print('Outbox is an empty frame')

# ----------Send thread

if thread.empty != True:
    db_connector_thread = DatabaseConnector(email_config.server, email_config.db_user, email_config.db_pass, email_config.database, 'thread')
    db_connector_thread.send(thread, 'thread')
    pre_existing_thread_updates = db_connector_thread.update_reply_thread(thread)
else:
    logging.info('Thread is an empty frame')
    print('Thread is an empty frame')

Current Subject Line There are no responses to the specified emails search criteria Local Supplier for Baseball Apparel in the inbox with a filter date of 3/4/2024
Current Subject Line You have 74 email responses to the specified search criteria Local Supplier for Baseball Apparel in the outbox with a filter date of 3/4/2024


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations_aws.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)
c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations_aws.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)


74 new emails appended to email_history
Thread is an empty frame
